In [1]:
import pandas as pd
import csv
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.utils import shuffle
import os

In [2]:
l1 = 'ita'
l2 = 'spa'

conf = 7

In [3]:
l1_freq_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/freq/{}'.format(l1)
l2_freq_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/freq/{}'.format(l2)

pl_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/filter/{0}_{1}.tsv'.format(l1, l2)

train_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/train/{0}_{1}.tsv'.format(l1, l2)
test_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/test/{0}_{1}.tsv'.format(l1, l2)

In [4]:
# Read l1 freq info

l1_ls = []
with open(l1_freq_fn) as f:
    lines = f.read().splitlines()
    for line in lines:
        l1_ls.append(line)

In [5]:
# 2519427
l1_f = pd.DataFrame({'ita' : l1_ls})
l1_f = l1_f.drop_duplicates(subset=('ita'))
l1_f['freq'] = l1_f.index
l1_f.head()
len(l1_f.index)

,ita,freq
0,</s>,0
1,",",1
2,.,2
3,-,3
4,',4


871053

In [6]:
# Read l2 freq info

l2_ls = []
with open(l2_freq_fn) as f:
    lines = f.read().splitlines()
    for line in lines:
        l2_ls.append(line)

In [7]:
# 985649
l2_f = pd.DataFrame({'spa' : l2_ls})
l2_f = l2_f.drop_duplicates(subset=('spa'))
l2_f['freq'] = l2_f.index
l2_f.head()
len(l2_f.index)

,spa,freq
0,de,0
1,</s>,1
2,",",2
3,.,3
4,la,4


985649

In [8]:
# Read panlex data
pl = pd.read_csv(pl_fn, sep='\t', header=None, names = [l1, l2, 'score'])
pl['score'] = pl.score.astype(int)
pl.head()
len(pl.index)

,ita,spa,score
0,dollaro,dólar,9
1,birr,birr,9
2,scellino,chelín,9
3,tongano,tongano,9
4,angolano,angoleño,9


219415

In [9]:
# Filter conf

pl = pl[pl.score >= conf]
len(pl.index)

51934

In [10]:
# Merge l1 freq info
pl_f = pd.merge(pl, l1_f, on=[l1])
pl_f.head()
len(pl_f.index)

,ita,spa,score,freq
0,dollaro,dólar,9,13136
1,dollaro,dólar,7,13136
2,dollaro,pavos,7,13136
3,dollaro,pavo,7,13136
4,birr,birr,9,123221


51934

In [11]:
# Merge l2 freq info
pl_ff = pd.merge(pl_f, l2_f, on=[l2])
pl_ff.head()
len(pl_ff.index)

,ita,spa,score,freq_x,freq_y
0,dollaro,dólar,9,13136,11555
1,dollaro,dólar,7,13136,11555
2,dollaro,pavos,7,13136,44297
3,dollaro,pavo,7,13136,20455
4,tacchino,pavo,8,48892,20455


51934

In [12]:
# Create bins from l1

bin1 = l1_f.iloc[1000:5000]
bin2 = l1_f.iloc[5000:20000]
bin3 = l1_f.iloc[20000:50000]
bin4 = l1_f.iloc[50000:100000]
bin5 = l1_f.iloc[100000:200000]

In [13]:
def get_300_uq(binN, alle, N, no_dup=True):
    print('Number of words in {0}: {1}'.format(N, len(binN.index)))
    
    alle_bin = pd.merge(alle, binN, on=[l1])
    print('len alle_bin: {0}'.format(len(alle_bin.index)))
        
    if no_dup:
        alle_bin = alle_bin.drop_duplicates(subset=[l1], keep=False)
        print('len alle_bin nodup: {0}'.format(len(alle_bin.index)))
    
    alle_bin = shuffle(alle_bin)
    
    testN = alle_bin.head(300)
    print('len testN: {0}'.format(len(testN.index)))

    return testN

In [14]:
test1 = get_300_uq(bin1, pl_ff, 1)

Number of words in 1: 4000
len alle_bin: 7268
len alle_bin nodup: 553
len testN: 300


In [15]:
test2 = get_300_uq(bin2, pl_ff, 2)

Number of words in 2: 15000
len alle_bin: 13374
len alle_bin nodup: 1841
len testN: 300


In [16]:
test3 = get_300_uq(bin3, pl_ff, 3)

Number of words in 3: 30000
len alle_bin: 12290
len alle_bin nodup: 2802
len testN: 300


In [17]:
test4 = get_300_uq(bin4, pl_ff, 4)

Number of words in 4: 50000
len alle_bin: 9385
len alle_bin nodup: 3030
len testN: 300


In [18]:
test5 = get_300_uq(bin5, pl_ff, 5)

Number of words in 5: 100000
len alle_bin: 6579
len alle_bin nodup: 2838
len testN: 300


In [19]:
# Concat test

frames = [test1, test2, test3, test4, test5]
pl_test = pd.concat(frames)
pl_test.head()
len(pl_test.index)

,ita,spa,score,freq_x,freq_y,freq
5405,riconosciuto,reconocido,7,3194,2306,3194
6567,costituito,compuesto,7,1496,833,1496
5926,fotografo,fotógrafo,7,4583,6628,4583
5906,conseguenza,consecuencia,7,1719,1615,1719
4598,commedia,comedia,7,1982,2260,1982


1500

In [21]:
# Create train without duplicates

test_ita = pl_test[[l1]].copy()
len(test_ita.index)

pl_train = pl_ff[~pl_ff.ita.isin(set(test_ita.ita))]
len(pl_ff.index)
len(pl_train.index)

pl_train = pl_train.sort_values(['freq_x', 'freq_y'], ascending=True)
pl_train_nodup = pl_train.drop_duplicates(subset=[l1], keep=False)
len(pl_train_nodup.index)

pl_train_nodup_5K = pl_train_nodup.head(5000)
pl_train_nodup_5K.head(10)
pl_train_nodup_5K.tail(10)
len(pl_train_nodup_5K.index)

1500

51934

50434

9696

,ita,spa,score,freq_x,freq_y
4102,",",",",8,1,2
5140,.,.,8,2,3
18299,del,para,7,14,25
3728,l,l,8,20,201
10683,fu,difunto,7,52,12726
47323,the,té,7,53,7471
4546,sua,su,7,56,22
51596,due,dos,7,57,45
4548,suo,su,7,61,22
2577,s,s,8,65,85


,ita,spa,score,freq_x,freq_y
40650,inorganico,inorgánico,7,58287,49775
31534,vaglia,giro,7,58320,4200
16697,purpureo,púrpura,7,58336,7523
22686,imperialista,imperialista,7,58342,42424
47422,tempia,sien,7,58347,71331
29968,metaforicamente,metafórico,7,58354,68515
51725,mutilazione,mutilación,7,58366,48580
39420,remunerazione,salarios,7,58393,12169
33851,frigio,frigio,7,58394,51099
44899,misericordioso,misericordioso,7,58445,72390


5000

In [82]:
# With duplicates

pl_train_nodup = pl_train
len(pl_train_nodup.index)

pl_train_nodup_5K = pl_train_nodup.head(5000)
pl_train_nodup_5K.head(10)
pl_train_nodup_5K.tail(10)
len(pl_train_nodup_5K.index)

72466

,eng,spa,score,freq_x,freq_y
18787,",",",",8,0,2
18785,",",.,8,0,3
18788,.,",",8,1,2
18786,.,.,8,1,3
27723,the,la,7,2,4
3660,the,el,7,2,6
27739,the,los,7,2,15
27734,the,las,7,2,20
27730,the,lo,7,2,34
12356,of,de,7,4,0


,eng,spa,score,freq_x,freq_y
4756,average,medio,7,749,303
58146,average,media,7,749,523
26061,average,común,7,749,669
58150,average,promedio,7,749,1865
21385,average,regular,7,749,2050
4075,average,corriente,7,749,2117
55777,average,mediocre,7,749,27773
20780,network,red,7,750,541
29697,network,emisora,7,750,5949
29696,network,interconectar,7,750,99091


5000

In [80]:
eng_te = set(pl_test[l1])
eng_tr = set(pl_train_nodup_5K[l1])
len(eng_te & eng_tr)

spa_te = set(pl_test[l2])
spa_tr = set(pl_train_nodup_5K[l2])
len(spa_te & spa_tr)

0

110

In [30]:
def save_tsv(df, fn):
    if not os.path.exists(os.path.dirname(fn)):
        os.makedirs(os.path.dirname(fn))
    pd.DataFrame.to_csv(df, columns=[l1, l2], header=False, path_or_buf=fn, sep='\t', index=False)

In [33]:
save_tsv(pl_train_nodup_5K, train_fn)
save_tsv(pl_test, test_fn)